In [7]:
import pandas as pd
import numpy as np

import requests
import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [9]:
address='Implerstr,Munich,Germany'
geolocator=Nominatim(user_agent='ishu')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('the latitude and longitude of the Implerstr, Munich, Germany is {},{}'.format(latitude,longitude))

the latitude and longitude of the Implerstr, Munich, Germany is 48.1201994,11.5484511


In [10]:
#foursquare details
CLIENT_ID = 'NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N' # your Foursquare ID
CLIENT_SECRET = 'WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ' # your Foursquare Secret
ACCESS_TOKEN = 'AWCDFCLJBBPUKSG2FX5TTMHSRG2OEIPTEQ5P3CH0DBSENWWX'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N
CLIENT_SECRET:WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ


In [170]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,latitude, longitude, VERSION, 1000, 100)

# send GET request and get trending venues
results = requests.get(url).json()["response"]['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5a8bf3b7b6eedb4f9cf1762d',
   'name': 'Bäckerei Neulinger',
   'location': {'address': 'Gotzinger Straße 48',
    'lat': 48.119164,
    'lng': 11.550225,
    'labeledLatLngs': [{'label': 'display',
      'lat': 48.119164,
      'lng': 11.550225}],
    'distance': 175,
    'postalCode': '81371',
    'cc': 'DE',
    'neighborhood': 'Sendling',
    'city': 'München',
    'state': 'Bayern',
    'country': 'Deutschland',
    'formattedAddress': ['Gotzinger Straße 48',
     '81371 München',
     'Deutschland']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 

In [171]:
venue_list=[]
venue_list.append([(
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for ven in venue_list for item in ven])
nearby_venues.columns = ['id',  
                  'Venue', 
                  'Venue_lat', 
                  'Venue_lon', 
                  'Venue_cat']

In [172]:
nearby_venues.head()

,id,Venue,Venue_lat,Venue_lon,Venue_cat
0,5a8bf3b7b6eedb4f9cf1762d,Bäckerei Neulinger,48.119164,11.550225,Bakery
1,4fd0b09ae4b0430cb62d82ee,Ćevabdžinica 10,48.119454,11.549473,Bosnian Restaurant
2,505c8ceee4b0bcb213237f07,Beirut Beirut,48.117450,11.548116,Falafel Restaurant
3,4b26a3d1f964a520d37e24e3,Piccola Italia,48.121067,11.545457,Italian Restaurant
4,4bd6c77129eb9c74708695e1,Ninh Restaurant,48.120994,11.544024,Vietnamese Restaurant


In [173]:
url='https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&oauth_token={}&v={}'.format('5a8bf3b7b6eedb4f9cf1762d',CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION)
res=requests.get(url).json()
res['response']['likes']['count']

7

In [174]:
def like(id):
    likes=[]
    
    for i in id:
        url='https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&oauth_token={}&v={}'.format(i,CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION)
        res=requests.get(url).json()
        likes.append(res['response']['likes']['count'])
    
    return(likes)      

In [175]:
likes=like(nearby_venues['id'])

In [176]:
nearby_venues['likes']=likes

In [177]:
nearby_venues.head()

,id,Venue,Venue_lat,Venue_lon,Venue_cat,likes
0,5a8bf3b7b6eedb4f9cf1762d,Bäckerei Neulinger,48.119164,11.550225,Bakery,7
1,4fd0b09ae4b0430cb62d82ee,Ćevabdžinica 10,48.119454,11.549473,Bosnian Restaurant,28
2,505c8ceee4b0bcb213237f07,Beirut Beirut,48.117450,11.548116,Falafel Restaurant,138
3,4b26a3d1f964a520d37e24e3,Piccola Italia,48.121067,11.545457,Italian Restaurant,23
4,4bd6c77129eb9c74708695e1,Ninh Restaurant,48.120994,11.544024,Vietnamese Restaurant,34


url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format('5a8bf3b7b6eedb4f9cf1762d',CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION)
res=requests.get(url).json()
res['response']['venue']['price']['tier']

def d(venue):
    infos=[]
    
    for i in venue:
        print(i)
        url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(i,CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION)
        res=requests.get(url).json()['response']
        
        infos.append([( 
            i,
            v['venue']['photos']['count'],
            v['venue']['tips']['count'], 
              
            v['venue']['price']['tier']) for v in results])
    
    nearby_venues = pd.DataFrame([i for info in infos for i in info])
    nearby_venues.columns = ['id',
                             'name',
                             'lat', 
                             'lon', 
                             'cat', 
                             'price_tier', 
                             'tips', 
                             'likes',
                             'people_liked',
                             'ratings']
    
    return(nearby_venues)   

data=d(nearby_venues['id'])

In [178]:
nearby_venues=nearby_venues.sort_values(by='likes',ascending=False).reset_index(drop=True)

In [179]:
me = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label,lk in zip(nearby_venues['Venue_lat'], nearby_venues['Venue_lon'], nearby_venues['Venue'],nearby_venues['likes']):
    label = folium.Popup((likes), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(me)  
    
me

In [180]:
nearby_venues['Venue_cat'].unique()

array(['Falafel Restaurant', 'Caucasian Restaurant', 'Rock Club',
       'Nightclub', 'Food Court', 'Burger Joint', 'Gastropub',
       'German Restaurant', 'Turkish Restaurant', 'Bar', 'Café', 'Pool',
       'Pizza Place', 'Pub', 'Supermarket', 'Middle Eastern Restaurant',
       'Gym', 'Bistro', 'Asian Restaurant', 'Hotel',
       'Vietnamese Restaurant', 'Comedy Club', 'Beer Bar',
       'Bosnian Restaurant', 'Restaurant', 'Plaza', 'Chinese Restaurant',
       'Italian Restaurant', 'Trattoria/Osteria', 'Cocktail Bar',
       'Spanish Restaurant', 'Seafood Restaurant', 'Japanese Restaurant',
       'Deli / Bodega', 'Market', 'Wine Shop', 'Butcher',
       'Comfort Food Restaurant', 'Drugstore', 'Doner Restaurant',
       'Greek Restaurant', 'Austrian Restaurant', 'Ice Cream Shop',
       'Szechuan Restaurant', 'Art Gallery', 'Grocery Store',
       'Laundry Service', 'Bakery', 'Organic Grocery', 'Candy Store',
       'Ethiopian Restaurant', 'Currywurst Joint',
       'Mediterranean R

In [182]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue_cat'].unique())))

There are 57 uniques categories.


In [196]:
a=nearby_venues[['Venue_cat','Venue','likes']]
a

,Venue_cat,Venue,likes
0,Falafel Restaurant,Beirut Beirut,138
1,Caucasian Restaurant,Iveria,130
2,Rock Club,Strøm,110
3,Nightclub,Substanz,87
4,Food Court,Alte Utting,74
...,...,...,...
95,Food & Drink Shop,Hong Kong Market,3
96,Supermarket,Edeka,2
97,Seafood Restaurant,Fisch Häusl,2
98,Mobile Phone Shop,Telekom Shop,0


# Analysing the Venues

In [191]:
num_top_venues = 5

for hood in nearby_venues['Venue_cat']:
    print("----"+hood+"----")
    temp = nearby_venues[nearby_venues['Venue_cat'] == hood].T.reset_index()
    print(temp)


----Falafel Restaurant----
       index                         0
0         id  505c8ceee4b0bcb213237f07
1      Venue             Beirut Beirut
2  Venue_lat                   48.1174
3  Venue_lon                   11.5481
4  Venue_cat        Falafel Restaurant
5      likes                       138
----Caucasian Restaurant----
       index                         1
0         id  4fe4bface4b043458f799d1a
1      Venue                    Iveria
2  Venue_lat                    48.125
3  Venue_lon                   11.5497
4  Venue_cat      Caucasian Restaurant
5      likes                       130
----Rock Club----
       index                         2
0         id  50c90690e4b03790c8f44f48
1      Venue                     Strøm
2  Venue_lat                   48.1248
3  Venue_lon                   11.5492
4  Venue_cat                 Rock Club
5      likes                       110
----Nightclub----
       index                         3                        12
0         id  4ade0cbdf9

5                        12  
----Comedy Club----
       index                        28
0         id  4ade0d0bf964a520756a21e3
1      Venue  Wirtshaus im Schlachthof
2  Venue_lat                   48.1235
3  Venue_lon                   11.5574
4  Venue_cat               Comedy Club
5      likes                        33
----Hotel----
       index                                23                        29  \
0         id          4bd773dc304fce72469433ab  4bc2add0920eb713ec911c2c   
1      Venue  Leonardo Hotel München City West       K+K Hotel am Harras   
2  Venue_lat                           48.1119                    48.117   
3  Venue_lon                           11.5476                   11.5382   
4  Venue_cat                             Hotel                     Hotel   
5      likes                                38                        33   

                         37                        80  
0  57237b8c498e8d0abedb9335  4bc78f8993bdeee1a88337ae  
1                B

       index                        56
0         id  4c1f1240b306c928b3f767b7
1      Venue              Magnus Bauch
2  Venue_lat                   48.1219
3  Venue_lon                   11.5583
4  Venue_cat                   Butcher
5      likes                        13
----Comfort Food Restaurant----
       index                        57
0         id  56364d48498e2c87615c7eb7
1      Venue               Drei Mühlen
2  Venue_lat                    48.121
3  Venue_lon                   11.5577
4  Venue_cat   Comfort Food Restaurant
5      likes                        13
----Italian Restaurant----
       index                        38                        42  \
0         id  4ade0cb5f964a520a46821e3  4b26a3d1f964a520d37e24e3   
1      Venue                  Friulana            Piccola Italia   
2  Venue_lat                   48.1259                   48.1211   
3  Venue_lon                   11.5537                   11.5455   
4  Venue_cat        Italian Restaurant        Italian R

5      likes                        53                         5
----Grocery Store----
       index                          72                        88
0         id    4fdc60f9e4b090d3f599a982  4d289002342d6dcbb1acf9ca
1      Venue  Güney Cavusoglu Supermarkt                 Löwenzahn
2  Venue_lat                     48.1217                   48.1269
3  Venue_lon                     11.5485                   11.5532
4  Venue_cat               Grocery Store             Grocery Store
5      likes                           9                         5
----Ethiopian Restaurant----
       index                        89
0         id  4cf69b63c28df04de8f3cb15
1      Venue                  Cafė Omo
2  Venue_lat                    48.126
3  Venue_lon                   11.5484
4  Venue_cat      Ethiopian Restaurant
5      likes                         5
----Currywurst Joint----
       index                        90
0         id  55771502498e91ef4b8492c3
1      Venue            Curry-Banditen
